<a href="https://colab.research.google.com/github/techfreckels/text/blob/master/HDBSCAN_ElectricityHubWholesale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import itertools
import datetime as dt
import statsmodels as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}
!pip install hdbscan
import hdbscan
plt.rcParams["figure.figsize"] = (16,12)

from IPython.display import HTML
css_str = '<style>ul {  list-style-type: none;  margin: 0;  padding: 0;}</style>'
html = HTML(css_str)
display(html)

# Unsupervised anomaly detection in electricity markets by HDBSCAN clustering method
HDBSCAN is a clustering algorithm developed by Campello, Moulavi, and Sander. It HDBSCAN stands for <i>Hierarchical Density-Based Spatial Clustering of Applications with Noise</i>.<br>

## No (few) assumptions except for some noise
We want to have as few assumptions about our data as possible. Perhaps the only assumptions that we can safely make are:
<ul>
<li>
    There is noise in our data
    </li>
    <li>
    There are clusters in our data which we hope to discover
    </li>
</ul> 

## Clustering dataset
To motivate our discussion, we start with a toy data set.
<img src="https://miro.medium.com/max/715/1*36x7yPCJGVrKnogBpE2n4w.png"/>
We can plot the data and identify 6 "natural" clusters in our dataset. We hope to automatically identify these through some clustering algorithm.
## K-means vs HDBSCAN
Knowing the expected number of clusters, we run the classical K-means algorithm and compare the resulting labels with those obtained using HDBSCAN.

<img src="https://miro.medium.com/max/2038/1*L-hr07E_ygPJEqDXgaoGQA.png"/>

Even when provided with the correct number of clusters, K-means clearly fails to group the data into useful clusters. HDBSCAN, on the other hand, gives us the expected clustering.

## Why does K-means fail?


Let us borrow a simpler example from ESLR [4] to illustrate how K-means can be sensitive to the shape of the clusters. Below are two clusterings from the same data. On the left, data was standardized before clustering. Without standardization, we get a “wrong” clustering.



Briefly, K-means performs poorly because the underlying assumptions on the shape of the clusters are not met; it is a parametric algorithm parameterized by the K cluster centroids, the centers of gaussian spheres. K-means performs best when clusters are:
<ul>
    <li>"round" or spherical</li>
    <li>equally sized </li>
    <li>equally dense </li>
    <li>most dense in the center of the sphere </li>
    <li>not contaminated by noise/outliers </li>
</ul>

Let us borrow a simpler example from ESLR to illustrate how K-means can be sensitive to the shape of the clusters. Below are two clusterings from the same data. On the left, data was standardized before clustering. Without standardization, we get a "wrong" clustering.

<img src="https://miro.medium.com/max/1662/1*gzRRGby6vq6buR1SlzGaJg.png" />


## What are the characteristics of our data?

We go back to our original data set and by simply describing it, it becomes obvious why K-means has a hard time. The data set has:
<ul>
    <li>
    Clusters with arbitrary shapes</li>
    <li>Clusters of different sizes</li>
    <li>Clusters with different densities</li>
    <li>Some noise and maybe some outliers</li>
</ul>
<img src="https://miro.medium.com/max/1408/1*nHCw-IeJvNWSm4iq0UkZNg.png" />

## Need robustness for data exploration

While each bullet point can be reasonably expected from a real-world dataset, each one can be problematic for parametric algorithms such as K-means. We might want to check if the assumptions of our algorithms are met before trusting their output. But, checking for these assumptions can be difficult when little is known about the data. This is unfortunate because one of the primary uses of clustering algorithms is data exploration where we are still in the process of understanding the data

Therefore, a clustering algorithm that will be used for data exploration needs to have as few assumptions as possible so that the initial insights we get are “useful”; having fewer assumptions make it more robust and applicable to a wider range of real-world data.


# Dense regions and multivariate modes
Now, we have an idea what type of data we are dealing with, let’s explore the core ideas of HDBSCAN and how it excels even when the data has:
<ul>
    <li>
    Arbitrarily shaped clusters</li>
    <li>Clusters with different sizes and densities</li>
    <li>Noise</li>

HDBSCAN uses a density-based approach which makes few implicit assumptions about the clusters. It is a non-parametric method that looks for a cluster hierarchy shaped by the multivariate modes of the underlying distribution. Rather than looking for clusters with a particular shape, it looks for regions of the data that are denser than the surrounding space. The mental image you can use is trying to separate the islands from the sea or mountains from its valleys.
    
## What’s a cluster?

How do we define a “cluster”? The characteristics of what we intuitively think as a cluster can be poorly defined and are often context-specific. (See Christian Hennig’s talk [5] for an overview)

If we go back to the original data set, the reason we identify clusters is that we see 6 dense regions surrounded by sparse and noisy space.
    
    <img src="https://miro.medium.com/max/1408/1*eStGcmNGVN3-WC2IcEDY4A.png" />
    
One way of defining a cluster which is usually consistent with our intuitive notion of clusters is: highly dense regions separated by sparse regions.

Look at the plot of 1-d simulated data. We can see 3 clusters.
  <img src="https://miro.medium.com/max/3243/1*xyD-oZmG6tGcAAXyxrn72g.png" />
    
## Looking at the underlying distribution

X is simulated data from a mixture of normal distributions, and we can plot the exact probability distribution of X.    
  <img src="https://miro.medium.com/max/1490/1*naEKid6E2eO43jgLsIhGmA.png" />
The peaks correspond to the densest regions and the troughs correspond to the sparse regions. This gives us another way of framing the problem assuming we know the underlying distribution, clusters are highly probable regions separated by improbable regions. Imagine the higher-dimensional probability distributions forming a landscape of mountains and valleys, where the mountains are your clusters.
    <img src="https://miro.medium.com/max/1490/1*W3kun_Pxbmgn6S_-ZzHZAA.png" />
    For those not as familiar, the two statements are practically the same:
<ul>
    <li>
    highly dense regions separated by sparse regions </li>
     <li>highly probable regions separated by improbable regions <li>
</ul>
One describes the data through its probability distribution and the other through a random sample from that distribution.

The PDF plot and the strip plot above are equivalent. PDF, probability density function, is interpreted as the probability at that point, or a small region around it, and when looking at a sample from X, it can also be interpreted as the expected density around that point.

Given the underlying distribution, we expect that regions that are more probable would tend to have more points (denser) in a random sample. Similarly, given a random sample, you can make inferences on the probability of a region based on the empirical density.
    
## Denser regions in the random sample correspond to more probable regions in the underlying distributions.

In fact, if we look at the histogram of a random sample of X, we see that it looks exactly like the true distribution of X. The histogram is sometimes called the empirical probability distribution, and with enough data, we expect the histogram to converge to the true underlying distribution.    
    
 <img src="https://miro.medium.com/max/1524/1*W75kS8rV_3sVmfOMojPDFA.png" />
Again, density = probability. Denser = more probable.

    
## But… what’s a cluster?

Sadly, even with our “mountains and valleys” definition of clusters, it can be difficult to know whether or not something is a single cluster. Look at the example below where we shifted one of the modes of X to the right. Although we still have 3 peaks, do we have 3 clusters? In some contexts, we might consider 3 clusters. “Intuitively” we say there are just 2 clusters. How do we decide?    

In [0]:
#@title
data = pd.read_csv("/home/mcfly/Documents/mari/ts_ercot.csv")
import plotly.offline as pyo
import plotly.graph_objs as go
pyo.init_notebook_mode()

#import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data["Date"],
    y=data["Price"],
    name="Price"       # this sets its legend entry
))

fig.update_layout(
    title="ERCOT",
    xaxis_title="Date",
    yaxis_title="Price",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="#7f7f7f"
    )
)

fig.show()

So we have a total of three clusters, with labels 0, 1, and 2. Importantly HDBSCAN is noise aware – it has a notion of data samples that are not assigned to any cluster. This is handled by assigning these samples the label -1. But wait, there’s more. The hdbscan library implements soft clustering, where each data point is assigned a cluster membership score ranging from 0.0 to 1.0. A score of 0.0 represents a sample that is not in the cluster at all (all noise points will get this score) while a score of 1.0 represents a sample that is at the heart of the cluster (note that this is not the spatial centroid notion of core). You can access these scores via the probabilities_ attribute.

In [0]:
a = []
x = []
y=[]
for i in range(0, len(data)-1):
    a.append([i,data["Price"][i]])    

X = a
X = StandardScaler().fit_transform(X)

for i in range(0,len(X)):
    x.append(X[i][0])
    y.append(X[i][1])

In [0]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=15).fit(X)
color_palette = sns.color_palette('deep', 8)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]

plt.scatter(*X.T, s=50, linewidth=0, c=cluster_member_colors, alpha=0.25)

In [0]:
#db = DBSCAN(eps=0.1, min_samples=3).fit(X)
#clusterer = hdbscan.HDBSCAN(min_cluster_size=15).fit(X)
hdb= hdbscan.HDBSCAN(min_samples=15,gen_min_span_tree=True).fit(X)
hcore_samples_mask = np.zeros_like(hdb.labels_, dtype=bool)
#core_samples_mask[db.core_sample_indices_] = True
hlabels = hdb.labels_
hn_clusters_ = len(set(hlabels)) - (1 if -1 in hlabels else 0)
hunique_labels = set(hlabels)

plt.figure(num=None, facecolor='w', edgecolor='k')

In [0]:
for k in hunique_labels:
    col=[0,0.5,1,1]
    if k == -1:
        col = [1, 0, 0, 1]
    hclass_member_mask = (hlabels == k)

    xy = X[hclass_member_mask & hcore_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', color=tuple(col),markersize=5, alpha=0.5)

    xy = X[hclass_member_mask & ~hcore_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', color=tuple(col), markersize=5, alpha=0.5)

plt.title('Estimated number of clusters: %d' % hn_clusters_)
plt.show()

In [0]:
sns.distplot(hdb.outlier_scores_[np.isfinite(clusterer.outlier_scores_)], rug=True)

In [0]:
#hdb.condensed_tree_.plot()

hdb.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette('deep', 16))

In [0]:
hdb.minimum_spanning_tree_.plot(edge_cmap='viridis',
                                      edge_alpha=0.6,
                                      node_size=80,
                                      edge_linewidth=2)

In [0]:
#hdb = hdbscan.HDBSCAN(min_cluster_size=15).fit(X)
color_palette = sns.color_palette('bright', 12)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in hdb.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, hdb.probabilities_)]
plt.scatter(*X.T, s=50, linewidth=0, c=cluster_member_colors, alpha=0.25)



In [0]:
threshold = pd.Series(hdb.outlier_scores_).quantile(0.9)
outliers = np.where(hdb.outlier_scores_ > threshold)[0]
plt.scatter(*X.T, s=50, linewidth=0, c='gray', alpha=0.25)
plt.scatter(*X[outliers].T, s=50, linewidth=0, c='red', alpha=0.5)

In [0]:
clusterer.single_linkage_tree_.plot()